In [1]:
import pandas as pd

In [158]:
climate = pd.read_csv("scored_v2/ft_meetings_climate_all_2.1.csv")
non_climate = pd.read_csv("scored_v2/ft_meetings_non_climate_all_2.1.csv")
frames = [climate, non_climate]
total = pd.concat(frames)

In [159]:
total.drop(['speaker', 'Unnamed: 0', 'index'], axis=1, inplace=True)

In [160]:
total['politician'] = total.politician.apply(lambda x: x.strip())

### Politicians that speak most about climate change related subjects

In [161]:
climate.drop(['speaker', 'Unnamed: 0', 'index'], axis=1, inplace=True)

In [162]:
climate

,politician,party,text,date,y_pred
0,Henrik Dam Kristensen,(S),"Jeg synes, det er lidt ærgerligt, at hr. Lars ...",2012-10-03,climate
1,Lars Christian Lilleholt,(V),Det får jo ganske alvorlige konsekvenser både ...,2012-10-03,climate
2,Ellen Trane Nørby,(V),"Jeg tror i hvert fald, at det ofte er sådan fo...",2012-10-04,climate
3,Magnus Heunicke,(S),"Altså, det er jo svært at blive enig, jeg tror...",2012-10-04,climate
4,Kisser Franciska Lehnert,(LA),Tak. Nu kalder Enhedslisten sig jo for et arbe...,2012-10-04,climate
...,...,...,...,...,...
6739,Peder Hvelplund,(EL),"Tak for det, formand. Jeg kan starte med at si...",2021-06-03,climate
6740,Magnus Heunicke,(S),"Tak for det, formand, og mange tak for den ove...",2021-06-03,climate
6741,Henrik Møller,(S),Tak. Efter sådan en historisk våd og regnfuld ...,2021-06-03,climate
6742,Torsten Gejl,(ALT),"Tak, formand. Jeg er glad for, at det ikke er ...",2021-06-04,climate


In [163]:
#Raw view 
climate.groupby(['politician', 'party']).count().sort_values(by='text', ascending=False)['text'].head(20)

politician                 party
Lars Christian Lilleholt   (V)      242
 Christian Poll            (ALT)    218
 Søren Egge Rasmussen      (EL)     195
 Rasmus Nordqvist          (ALT)    177
 Jens Joel                 (S)      165
 Mikkel Dencker            (DF)     144
 Maria Reumert Gjerding    (EL)     132
 Pia Olsen Dyhr            (SF)     129
 Thomas Danielsen          (V)      124
 Lisbeth Bech Poulsen      (SF)     114
Torsten Gejl               (ALT)    111
 Josephine Fock            (ALT)     96
 Mette Abildgaard          (KF)      93
 Henning Hyllested         (EL)      84
 Carsten Bach              (LA)      82
Dan Jørgensen              (S)       80
 Per Clausen               (EL)      78
 Signe Munk                (SF)      68
Signe Munk                 (SF)      67
 Ole Birk Olesen           (LA)      64
Name: text, dtype: int64

In [165]:
# Taking out some the smaller parties
parties = ['(S)', '(V)', '(LA)', '(EL)', '(DF)', '(RV)', '(KF)', '(SF)', '(ALT)']

small_parties = ['(NQ)', '(NB)', '(JF)', '(SIU)', '(SP)', '(T)', '(UFG)', '(IA)', '(KD)']

total = total[total.party.isin(parties) == True]

total.party.unique()

array(['(S)', '(V)', '(LA)', '(EL)', '(DF)', '(RV)', '(KF)', '(SF)',
       '(ALT)'], dtype=object)

In [166]:
total['climate'] = total.apply(lambda x: 1 if x['y_pred'] == 'climate' else 0, axis=1)
total['non_climate'] = total.apply(lambda x: 1 if x['y_pred'] == 'non-climate' else 0, axis=1)

In [167]:
total

,politician,party,text,date,y_pred,climate,non_climate
0,Henrik Dam Kristensen,(S),"Jeg synes, det er lidt ærgerligt, at hr. Lars ...",2012-10-03,climate,1,0
1,Lars Christian Lilleholt,(V),Det får jo ganske alvorlige konsekvenser både ...,2012-10-03,climate,1,0
2,Ellen Trane Nørby,(V),"Jeg tror i hvert fald, at det ofte er sådan fo...",2012-10-04,climate,1,0
3,Magnus Heunicke,(S),"Altså, det er jo svært at blive enig, jeg tror...",2012-10-04,climate,1,0
4,Kisser Franciska Lehnert,(LA),Tak. Nu kalder Enhedslisten sig jo for et arbe...,2012-10-04,climate,1,0
...,...,...,...,...,...,...,...
198664,Mattias Tesfaye,(S),"Jeg var udlændingeordfører, da den daværende r...",2021-06-24,non-climate,0,1
198665,Kristian Hegaard,(RV),"Tak for det, og tak til ministeren for at gå o...",2021-06-24,non-climate,0,1
198666,Mattias Tesfaye,(S),Jeg har bare lige en enkelt lille korrektion: ...,2021-06-24,non-climate,0,1
198667,Kristian Hegaard,(RV),Men når nu vi har haft en noget unuanceret deb...,2021-06-24,non-climate,0,1


In [171]:
total_grouped = total.groupby(['politician', 'party'], as_index=False)[['climate','non_climate']].sum()

total_grouped_party = total.groupby(['party'], as_index=False)[['climate', 'non_climate']].sum()

In [169]:
total_grouped['percentage_climate'] = total_grouped.apply(lambda x: x['climate']/(x.climate + x.non_climate), axis=1)

In [172]:
total_grouped_party['percentage_climate'] = total_grouped_party.apply(lambda x: x['climate']/(x.climate + x.non_climate), axis=1)

### Looking at how the Parties talk about Climate Change

In [175]:
total_grouped_party.sort_values(by='percentage_climate', ascending=False)

,party,climate,non_climate,percentage_climate
0,(ALT),828,7364,0.101074
5,(RV),571,13195,0.041479
2,(EL),1040,24659,0.040469
7,(SF),632,15207,0.039902
8,(V),1136,35694,0.030844
4,(LA),400,14330,0.027155
6,(S),1069,40109,0.025960
3,(KF),310,11681,0.025853
1,(DF),510,27950,0.017920


In [170]:
total_grouped[total_grouped.climate>20].sort_values(by='percentage_climate', ascending=False).head(20)

,politician,party,climate,non_climate,percentage_climate
322,Ruben Kidde,(RV),37,29,0.560606
330,Signe Munk,(SF),135,107,0.557851
15,Anne Paulin,(S),58,78,0.426471
193,Lars Christian Lilleholt,(V),301,565,0.347575
219,Mai Villadsen,(EL),74,155,0.323144
254,Mikkel Dencker,(DF),152,330,0.315353
58,Christian Poll,(ALT),218,662,0.247727
356,Thomas Danielsen,(V),124,506,0.196825
228,Maria Reumert Gjerding,(EL),132,547,0.194404
363,Tommy Ahlers,(V),40,209,0.160643


In [136]:
lst = total_grouped.party

In [147]:
lst = lst[lst.str.len()>5]
lst_p = []
for val in lst:
    lst_p.append(val)
    
lst_p

['Samuelsen',
 'Halsboe-Jørgensen',
 'Jørgensen',
 'Wammen',
 'Mortensen',
 'Hækkerup',
 'Rasmussen',
 'Vestager',
 'Tesfaye',
 'Gjerskov',
 'Kollerup',
 'Hækkerup',
 'Poulsen',
 'Pedersen',
 'Søvndal']

In [148]:
total_grouped[total_grouped.party.isin(lst_p)]

,politician,party,climate,non_climate,percentage_climate
7,Anders Samuelsen,Samuelsen,1,14,0.066667
11,Ane Halsboe-Jørgensen Ane Halsboe-Jørgensen,Halsboe-Jørgensen,0,5,0.000000
70,Dan Jørgensen Dan Jørgensen,Jørgensen,1,36,0.027027
85,Europaministeren Nicolai Wammen,Wammen,0,9,0.000000
94,Flemming Møller Mortensen Flemming Møller Mort...,Mortensen,1,0,1.000000
168,Karen Hækkerup Karen Hækkerup,Hækkerup,0,8,0.000000
205,Lars Løkke Rasmussen,Rasmussen,0,15,0.000000
238,Margrethe Vestager Margrethe Vestager,Vestager,0,1,0.000000
251,Mattias Tesfaye Mattias Tesfaye,Tesfaye,0,10,0.000000
261,Mette Gjerskov,Gjerskov,0,88,0.000000


### There seem to exist duplicates in our data for some of the politicians


In [125]:
total_grouped[total_grouped.politician.str.contains('Dan')]

,politician,party,climate,non_climate,percentage_climate
69,Dan Jørgensen,(S),80,682,0.104987
70,Dan Jørgensen Dan Jørgensen,Jørgensen,1,36,0.027027
71,Daniel Rugholm,(KF),0,79,0.000000
72,Daniel Toft Jakobsen,(S),0,106,0.000000
73,Danny K. Malkowski,(LA),2,33,0.057143
376,Thomas Danielsen,(V),124,506,0.196825


In [113]:
total_grouped[total_grouped.duplicated('politician')]

total_grouped[total_grouped.politician.str.contains('Dan Jørgensen')]



0             Alex Ahrendtsen 
1             Alex Vanopslagh 
2       Anders G. Christensen 
3            Anders Johansson 
4             Anders Kronborg 
                ...           
652        Villum Christensen 
653              Villy Søvndal
654                 Vivi Kier 
655              Zenia Stampe 
656          Özlem Sara Cekic 
Name: politician, Length: 657, dtype: object